一些预处理，主要是让eta_schedule中没有空行.

In [10]:
import numpy as np
import pandas as pd

task_df = pd.read_csv("../database/data/eta_task.csv")
schedule_df = pd.read_csv("../database/data/eta_task_schedule.csv")
schedule_df.index = [False] * len(schedule_df)
schedule_df

,t,path,roads,weights
False,9,"[(116.4905155, 39.8940372), (116.4883361, 39.8...","[37956, 32078, 32074, 34881, 31134, 14277, 220...","[186.298, 371.614, 225.193, 207.991, 169.829, ..."
False,11,"[(116.4513528, 39.9436912), (116.4493035, 39.9...","[13123, 24303, 24279, 30808, 378, 21507, 35021...","[174.719, 133.346, 86.412, 145.256, 9.743, 136..."
False,13,"[(116.3403829, 39.8001964)]",[],[]
False,16,"[(116.4788573, 39.8006971), (116.4782662, 39.8...","[25565, 25577, 25583, 25594, 25563, 25587]","[95.739, 113.064, 413.376, 439.087, 456.12, 34..."
False,24,"[(116.2505558, 39.9025875), (116.2506942, 39.9...","[36347, 36350, 29455, 8271, 20563, 33390, 2056...","[11.806, 280.225, 398.223, 373.027, 198.087, 5..."
...,...,...,...,...
False,21983,"[(116.4299934, 39.8870815), (116.4339522, 39.8...","[12398, 12458, 12338, 12440, 5171, 23753, 1497...","[394.861, 195.611, 51.109, 105.008, 294.605, 9..."
False,21984,"[(116.4353636, 39.8974426), (116.4365734, 39.8...","[12174, 14895, 14966, 14932, 20054, 14949, 149...","[191.571, 41.384, 85.807, 59.815, 43.23, 193.9..."
False,21991,"[(116.3747006, 39.910886), (116.3716227, 39.91...","[10448, 4554, 4992, 10575, 4990, 10292, 10319,...","[262.587, 112.787, 126.826, 17.826, 49.909, 90..."
False,21997,"[(116.3246973, 39.9371366), (116.3236868, 39.9...","[19332, 16122, 14356, 14471, 14496, 14491, 225...","[90.173, 230.948, 121.11, 9.134, 34.11, 108.29..."


In [11]:
for i in range(len(schedule_df)):
    if pd.isnull(schedule_df.iloc[i]['path']):
        schedule_df.iloc[i,1] = str([(-1,-1)])
        schedule_df.iloc[i,2] = str([])
        schedule_df.iloc[i,3] = str([])
schedule_df.to_csv("../database/data/eta_schedule_noempty.csv", index=False)

In [ ]:
schedule_df = pd.read_csv("../database/data/eta_schedule_noempty.csv")
for r in schedule_df.iterrows():
    print(r[1][2])

进行预测

In [1]:
import numpy as np
import pandas as pd
import torch
from Dataset import ETATaskData, RoadFeatures
from GRU import GRUmodel, SimpleGRUBagging, GRUmodelBoosting
from Utils import *

In [3]:
#nbags = 6   # 筛选出来6个
nboosting = 9

trajSet = ETATaskData("../database/data/eta_schedule_noempty.csv", "../database/data/eta_task.csv", 8)
rawroadfeat = RoadFeatures("./road_features_with_lengths.pkl", "../database/data/road.csv")
#base_bagging_dir = "./GRUBagging"
base_dir = "./newBoosting/"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# GRUs = SimpleGRUBagging(rawroadfeat.n_features, device, nbags, is_training=False, params_path=base_dir).to(device)
# fundGRU = GRUmodel(rawroadfeat.n_features, 128, device).eval().to(device)
# fundGRU.load_state_dict(torch.load("./GRUonly_result/rawfeature_reg_gru.pt"))
GRUs = GRUmodelBoosting(rawroadfeat.n_features, device, 9, params_path=base_dir)

ret = np.zeros((len(trajSet),2))
l = 0
for tids, roadids, tids, start_end_points, start_speeds, hour, holiday in trajSet.iter_by_order(hour_holiday=True):
    #fund = fundGRU.predict(roadids, start_end_points, start_speeds, rawroadfeat)
    #offset = GRUs.predict(roadids, start_speeds, rawroadfeat)
    #t = torch.clamp(fund + offset, 2).detach().cpu().numpy()
    t = GRUs.predict(roadids, start_end_points, start_speeds, rawroadfeat, hour, holiday).detach().cpu().numpy()
    ret[l:l+t.shape[0], 0] = tids
    ret[l:l+t.shape[0], 1] = t
    l += t.shape[0]

df = pd.DataFrame(ret, columns=['tid', 'timd'])
df.to_csv("../database/data/eta_answer.csv", index=False)

d:\classes\Datamining\BUAA-DM23\ETA\Dataset.py:156: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.road_ids = [ast.literal_eval(r[1][2]) for r in schedule_df.iterrows()]   # 注意iterrows()的结果是一个元组，第一项是idx!
d:\classes\Datamining\BUAA-DM23\ETA\Dataset.py:158: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  paths = [ast.literal_eval(r[1][1]) for r in schedule_df.iterrows()]


In [13]:
# 整理一下answer.
import datetime

fmt = '%Y-%m-%dT%H:%M:%SZ'
tid2predict = {}
for i in range(ret.shape[0]):
    if ret[i,0]==0:   # 剩下的无效了.
        break
    tid2predict[int(ret[i,0])] = ret[i,1]

df_etatask = pd.read_csv("../database/data/eta_task.csv")
df_start_info = df_etatask[df_etatask.index%2==0]  # 只留起点那一行.
ans = []
for idx, r in df_start_info.iterrows():
    tid = int(r[1])
    start_time = r[2]
    entity_id = r[3]
    speed = r[6]
    holiday = r[7]
    try:
        m = tid2predict[tid]
        _start = datetime.datetime.strptime(start_time, fmt)
        delta = datetime.timedelta(minutes=m)
        _end = _start + delta
        end_time = _end.strftime(fmt)
    except Exception as e:
        end_time = '----'
        m = -1
    ans.append([tid, entity_id, speed, holiday, start_time, end_time, m])

df = pd.DataFrame(ans, columns=['traj_id', 'entity_id', 'start_speed', 'holidays', 'start_time', 'final_time', 'duration(min)'])
df.to_csv("./Result/eta_task_answer.csv", index=False)

C:\Users\admin\AppData\Local\Temp\ipykernel_21568\1326256200.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tid = int(r[1])
C:\Users\admin\AppData\Local\Temp\ipykernel_21568\1326256200.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  start_time = r[2]
C:\Users\admin\AppData\Local\Temp\ipykernel_21568\1326256200.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  entity_id = r[3]
C:\Users\admin\AppData\Local\Temp\ipykernel